# Imports

In [239]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Lambda, BatchNormalization

# Loading data

In [240]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
X = X.reshape(-1, 8, 8)  # Reshape to (1797, 8, 8)
X = np.expand_dims(X, axis=-1)  # Shape becomes (1797, 8, 8, 1)

y = digits.target
# Print the shape of the data
print(f'Shape of X (features): {X.shape}')  # (1797, 64)
print(f'Shape of y (labels): {y.shape}')    # (1797,)

# Number of occurances of each label 
unique_digits, counts = np.unique(y, return_counts=True)
digit_counter = {}
for i in range(len(unique_digits)):
    digit_counter[unique_digits[i]] = counts[i]




Shape of X (features): (1797, 8, 8, 1)
Shape of y (labels): (1797,)


# Creating pairs

In [241]:
def create_balanced_pairs(X, y, unique_labels, num_pairs=1000):
    """Creates an equal number of positive (same class) and negative (different class) pairs."""
    pairs = []
    labels = []
    
    half_pairs = num_pairs // 2  # Half positive, half negative
    
    # Step 1: Create positive pairs
    for _ in range(half_pairs):
        # Randomly select a class
        digit_class = np.random.choice(unique_labels)
        
        # Get indices for this class
        indices = np.where(y == digit_class)[0]
        
        # Select 2 random indices from the same class
        if len(indices) >= 2:
            idx1, idx2 = np.random.choice(indices, size=2, replace=False)
            image1 = X[idx1]
            image2 = X[idx2]
            
            # Add the pair and label it as 1 (positive pair)
            pairs.append([image1, image2])
            labels.append(0.)
    
    # Step 2: Create negative pairs
    for _ in range(half_pairs):
        # Randomly select two different classes
        class1, class2 = np.random.choice(unique_labels, size=2, replace=False)
        
        # Get indices for each class
        indices1 = np.where(y == class1)[0]
        indices2 = np.where(y == class2)[0]
        
        # Select one image from each class
        idx1 = np.random.choice(indices1)
        idx2 = np.random.choice(indices2)
        
        image1 = X[idx1]
        image2 = X[idx2]
        
        # Add the pair and label it as 0 (negative pair)
        pairs.append([image1, image2])
        labels.append(1.)
    
    # Convert pairs and labels to numpy arrays
    return np.array(pairs), np.array(labels)

In [242]:
pairs_train, labels_train = create_balanced_pairs(X, y, [0, 1, 2, 3, 4, 5, 6], 50000)
pairs_test, labels_test = create_balanced_pairs(X, y, [7, 8, 9], 10000)
pairs_train.shape

(50000, 2, 8, 8, 1)

# Visualize data

In [243]:
# Function to visualize a single pair of images and its label
def plot_image_pair(pair, label):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    # Unpack the pair into two images
    image1, image2 = pair

    # Plot the first image in the pair
    axes[0].imshow(image1.reshape(8, 8), cmap='gray')
    axes[0].set_title('Image 1', fontsize=12)
    axes[0].axis('off')

    # Plot the second image in the pair
    axes[1].imshow(image2.reshape(8, 8), cmap='gray')
    axes[1].set_title('Image 2', fontsize=12)
    axes[1].axis('off')

    # Display whether the pair is positive or negative
    pair_label = 'Positive Pair' if label == 1 else 'Negative Pair'
    plt.suptitle(pair_label, fontsize=16)
    plt.show()

# Visualize some of the training pairs
for i in range(-1, -10, -1):  # Change this to display more or fewer pairs
    pair = pairs_test[i]  # Get the pair of images
    label = labels_test[i]  # Get the corresponding label (1 for positive, 0 for negative)
    
    # Plot the pair with its label (positive or negative)
#    plot_image_pair(pair, label)

# Build CNN Part

In [244]:
cnn_part = Sequential()
cnn_part.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(8,8,1)))
cnn_part.add(MaxPooling2D())
cnn_part.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
#cnn_part.add(BatchNormalization())
cnn_part.add(Flatten())
cnn_part.add(Dense(units=128, activation="relu"))

In [245]:
cnn_part.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 6, 6, 64)          640       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 3, 3, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 1, 1, 128)         73856     
                                                                 
 flatten_14 (Flatten)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 128)               16512     
                                                                 
Total params: 91,008
Trainable params: 91,008
Non-trainable params: 0
_________________________________________________

# Build SNN

In [246]:
input1, input2 = Input(shape=(8,8,1), name="Image1"), Input(shape=(8,8,1), name="Image2")
# Share the single output layer for both inputs
output1, output2 = cnn_part(input1), cnn_part(input2)

In [247]:
from tensorflow.keras import backend as K

# Creating the final layer of the SNN
def euclidean_distance(vectors):
    # vectors is a list containing two tensors
    x, y = vectors
    # Compute the Euclidean distance between x and y
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def contrastive_loss(y_true, y_pred):
    """
    y_true: Binary labels (1 for similar pairs, 0 for dissimilar pairs)
    y_pred: Euclidean distances between the output embeddings
    """
    margin = 1.0  # This can be tuned
    positive_loss = y_true * K.square(y_pred)  # Loss for similar pairs
    negative_loss = (1 - y_true) * K.square(K.maximum(margin - y_pred, 0))  # Loss for dissimilar pairs
    return K.mean(positive_loss + negative_loss)  # Mean loss for the batch

In [248]:
final_output = Lambda(euclidean_distance, output_shape=(1,))([output1, output2])
#final_output = Dense(1, activation='sigmoid')(distance_layer)

In [249]:
snn = Model((input1, input2), final_output)

In [250]:
from tensorflow.keras.optimizers import Adam

snn.compile(optimizer=Adam(),
              loss=contrastive_loss,  # Loss function for classification
              metrics=['accuracy']
              )

In [251]:
snn.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Image1 (InputLayer)            [(None, 8, 8, 1)]    0           []                               
                                                                                                  
 Image2 (InputLayer)            [(None, 8, 8, 1)]    0           []                               
                                                                                                  
 sequential_14 (Sequential)     (None, 128)          91008       ['Image1[0][0]',                 
                                                                  'Image2[0][0]']                 
                                                                                                  
 lambda_14 (Lambda)             (None, 1)            0           ['sequential_14[0][0]',   

# Train SNN

In [252]:
X1_train = np.array([pair[0] for pair in pairs_train])  # First image in each pair
X2_train = np.array([pair[1] for pair in pairs_train])  # Second image in each pair

X1_test = np.array([pair[0] for pair in pairs_test])  # First image in each pair
X2_test = np.array([pair[1] for pair in pairs_test])  # Second image in each pair

In [253]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3)
snn.fit(x=[X1_train, X2_train], 
        y = labels_train, 
        batch_size = 32, 
        epochs=100,
        validation_data=([X1_test, X2_test], labels_test),
        callbacks=[es],
        shuffle=True)

Epoch 1/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2826 - accuracy: 0.5107 - val_loss: 0.3277 - val_accuracy: 0.6288
Epoch 2/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2667 - accuracy: 0.4974 - val_loss: 0.3231 - val_accuracy: 0.6399
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2626 - accuracy: 0.4874 - val_loss: 0.3277 - val_accuracy: 0.6578
Epoch 4/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2639 - accuracy: 0.4857 - val_loss: 0.3461 - val_accuracy: 0.6363
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2613 - accuracy: 0.4821 - val_loss: 0.3754 - val_accuracy: 0.6704


In [254]:
y_predict = snn.predict([X1_test,X2_test])
from sklearn.metrics import accuracy_score
Y_predict_decisions = []
for i in range(len(y_predict)):
    if y_predict[i] > 0.5:
        Y_predict_decisions.append(1)
    else:
        Y_predict_decisions.append(0)

# Calculate accuracy
accuracy = accuracy_score(labels_test, Y_predict_decisions)
print("Accuracy:", accuracy)


313/313 [==============================] - 1s 1ms/step
Accuracy: 0.6704
